# Experiment #1

Let's start with a simple scenario... the wind velocity rotates in a circle and the agent uses uniform probability functions bounded to a fairly tight range.

* Winds: *Oscillatory*
    * Description of the wind behavior: the wind velocity vector traces out a circle in the XY plane blowing at a constant speed of 10 m/s and changing direction at a rate of ~57 degrees (1 radian) per second.
    * base_velocity = (0, 0, 0) m/s
    * amplitude = (10, 10, 0) m/s
    * frequency = (1, 1, 0) rad/s
    * phase_offset = (0, pi/2, 0) rad
* Aiming geometry: *Spherical*
* Agent probability distributions: *Uniform* (for timing, speed, & aiming)
    * timing: [0, 2] s
    * speed: [40, 44] m/s
    * aiming:
        * Theta (X1): [-10, 10] deg --> [-0.1745329, 0.1745329] rad
        * Phi (X2): [42, 48] deg --> [0.7330383, 0.837758] rad
* Simulation sampling:
    * num_runs = 10,000
    * num_workers = 1

## Imports

In [1]:
from simulator.simulation.scientists import ExperimentRunner, ExperimentCollater

import numpy as np
import requests

## Step 1: Generate Winds Spacetime

In [2]:
windgenparams = {
    'is_oscillatory': True,
    'amplitude': [10, 10, 0],
    'frequency': [1, 1, 0],
    'phase_offset': [0, np.pi, 0],
}
res = requests.post('http://localhost:8000/winds/wind-gen-params/', data=windgenparams)
res_data = res.json()
print(res_data)
windgenparams_id = res_data['id']
print(windgenparams_id)

{'message': 'Already exists', 'id': 'a7f87aeb-07ff-4ff9-9697-063621bc11f2'}
a7f87aeb-07ff-4ff9-9697-063621bc11f2


In [3]:
windspacetime = {
    'generator_name': 'oscillatory',
    'generator_params': windgenparams_id,
    'duration': 10,
    'timestep': 0.01,
}
res = requests.post('http://127.0.0.1:8000/winds/wind-spacetimes/', data=windspacetime)
res_data = res.json()
print(res_data)
windspacetime_id = res_data['id']
print(windspacetime_id)

{'message': 'Already exists', 'id': 'bff004b0-53c1-49a6-be2b-5a1006e6339e'}
bff004b0-53c1-49a6-be2b-5a1006e6339e


## Step 2: Run Simulation Experiment

In [1]:
experiment_params = {
        'windgenparams_id': windgenparams_id,
        'windspacetime_id': windspacetime_id,
        'num_trials': 10000,
        'timestep': 0.01,
        'prob_speed_fn_name': 'Uniform',
        'prob_speed_min': 40,
        'prob_speed_max': 44, # m/s
        'prob_speed_center': None,
        'prob_speed_spread': None,
        'prob_timing_fn_name': 'Uniform',
        'prob_timing_min': 0, # s
        'prob_timing_max': 2,
        'prob_timing_center': None,
        'prob_timing_spread': None,
        'prob_aiming_fn_name': 'Uniform',
        'prob_aiming_geometry': 'Spherical',
        'prob_aiming_X1_min': -0.1745329,  # -10 deg
        'prob_aiming_X1_max': 0.1745329,   #  10 deg
        'prob_aiming_X1_center': None,
        'prob_aiming_X1_spread': None,
        'prob_aiming_X2_min': 0.7330383,   # 42 deg
        'prob_aiming_X2_max': 0.837758, # 48 deg
        'prob_aiming_X2_spread': None,
        'prob_aiming_X3_min': 0,      # throwaway coordinate
        'prob_aiming_X3_max': 1,
        'prob_aiming_X3_center': None,
        'prob_aiming_X3_spread': None,    
}

runner = ExperimentRunner(experiment_params)

simtrial_ids = runner.run_experiment()

collater = ExperimentCollater(experiment_params, chunked_simtrial_ids=[simtrial_ids])
collater.save_experiment()

NameError: name 'windgenparams_id' is not defined

## Step 3: Analyze Results